Ignore this file now, it's old

Start by getting more data.

Include both snotel and weather station data.

In [57]:
import pandas as pd

def get_seasonal_nwac():
    crystal_summit_seasonal = pd.read_csv('../content/csv/seasonal_weather_crystal_summit.csv')
    # todo: aggregate from multiple sources
    return crystal_summit_seasonal

seasonal_nwac = get_seasonal_nwac()
seasonal_nwac.head()

,Date PDT,Temp F,RH %,Min mph,Spd mph,Gust mph,Dir deg
0,Apr 24 6:00,23,98,7,18,31,W
1,Apr 24 5:00,23,98,6,19,30,W
2,Apr 24 4:00,23,98,7,16,29,W
3,Apr 24 3:00,23,98,4,12,21,W
4,Apr 24 2:00,25,97,0,2,9,NW


In [58]:
seasonal_nwac.tail()

,Date PDT,Temp F,RH %,Min mph,Spd mph,Gust mph,Dir deg
5642,Sep 1 4:00,57,46,10,17,23,W
5643,Sep 1 3:00,57,43,9,16,23,W
5644,Sep 1 2:00,59,41,11,19,27,W
5645,Sep 1 1:00,59,37,8,17,23,W
5646,Sep 1 0:00,59,36,6,15,23,W


In [59]:
def get_seasonal_snotel():
    morse_lake_seasonal = pd.read_csv('../content/csv/seasonal_snotel_morse_lake.csv')
    # todo: aggregate from multiple sites
    return morse_lake_seasonal

seasonal_snotel = get_seasonal_snotel()
seasonal_snotel.head()

,Date PDT,Temp F,PcpAc in,∆Pcp in,SWE in,∆SWE in,SnoHt in,∆SnoHt in
0,Apr 24 6:00,30.0,NaN,NaN,57.0,-0.5,126.0,1.0
1,Apr 24 5:00,30.0,NaN,NaN,57.1,-0.2,126.0,1.0
2,Apr 24 4:00,31.0,NaN,NaN,57.2,0.0,123.0,-2.0
3,Apr 24 3:00,31.0,NaN,NaN,57.2,0.2,122.0,-3.0
4,Apr 24 2:00,31.0,NaN,NaN,57.2,0.3,124.0,-2.0


In [60]:
seasonal_snotel.tail()

,Date PDT,Temp F,PcpAc in,∆Pcp in,SWE in,∆SWE in,SnoHt in,∆SnoHt in
5593,Sep 1 5:00,53.0,86.3,0.0,NaN,NaN,0.0,0.0
5594,Sep 1 4:00,57.0,86.3,0.0,NaN,NaN,0.0,0.0
5595,Sep 1 3:00,62.0,86.3,0.0,NaN,NaN,0.0,0.0
5596,Sep 1 2:00,58.0,86.3,0.0,NaN,NaN,0.0,0.0
5597,Sep 1 1:00,58.0,86.3,0.0,0.0,0.0,0.0,0.0


todo: what happens if I drop columns I don't think are useful? How does that affect the prediciton accuracy? How does that align with my knowledge of snow science?

In [61]:
seasonal_nwac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5647 entries, 0 to 5646
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date PDT  5647 non-null   object
 1   Temp F    5647 non-null   int64 
 2   RH %      5647 non-null   int64 
 3   Min mph   5647 non-null   int64 
 4   Spd mph   5647 non-null   int64 
 5   Gust mph  5647 non-null   int64 
 6   Dir deg   5647 non-null   object
dtypes: int64(5), object(2)
memory usage: 308.9+ KB


In [62]:
seasonal_snotel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5598 entries, 0 to 5597
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date PDT   5598 non-null   object 
 1   Temp F     5597 non-null   float64
 2   PcpAc in   5068 non-null   float64
 3   ∆Pcp in    5047 non-null   float64
 4   SWE in     5290 non-null   float64
 5   ∆SWE  in   5266 non-null   float64
 6   SnoHt in   4800 non-null   float64
 7   ∆SnoHt in  4417 non-null   float64
dtypes: float64(7), object(1)
memory usage: 350.0+ KB


Join the dataframes (should have same date)

In [63]:
def get_seasonal_weather(nwac_df, snotel_df):
    # Join the two dataframes
    seasonal_weather = nwac_df.join(snotel_df, lsuffix=" NWAC", rsuffix=" SNOTEL")
    # Reconcile dates
    try:
        seasonal_weather = seasonal_weather.drop('Date PDT SNOTEL', axis=1)
        seasonal_weather = seasonal_weather.rename(columns={'Date PDT NWAC': 'Date PDT'})
    except Exception as err:
        print('Error:', err)
    # Reconcile temperature
    try:
        mean_temp = seasonal_weather.drop(['Temp F NWAC', 'Temp F SNOTEL'], axis=1).mean()
        seasonal_weather['Temp F'] = mean_temp
    except Exception as err:
        print('Error:', err)
    try:
        seasonal_weather = seasonal_weather.drop('Date', axis=1)
    except Exception as err:
        print('Error:', err)
    # Return
    return seasonal_weather

seasonal_weather = get_seasonal_weather(seasonal_nwac, seasonal_snotel)
seasonal_weather.iloc[100:105]

Error: "['Date'] not found in axis"


/tmp/ipykernel_27752/1993969672.py:12: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  mean_temp = seasonal_weather.drop(['Temp F NWAC', 'Temp F SNOTEL'], axis=1).mean()


,Date PDT,Temp F NWAC,RH %,Min mph,Spd mph,Gust mph,Dir deg,Temp F SNOTEL,PcpAc in,∆Pcp in,SWE in,∆SWE in,SnoHt in,∆SnoHt in,Temp F
100,Apr 20 2:00,16,94,0,6,12,WSW,23.0,NaN,NaN,54.2,0.4,129.0,2.0,NaN
101,Apr 20 1:00,17,95,2,6,13,WSW,24.0,NaN,NaN,54.2,0.4,129.0,3.0,NaN
102,Apr 20 0:00,17,94,0,6,13,WSW,24.0,NaN,NaN,54.2,0.4,130.0,3.0,NaN
103,Apr 19 23:00,18,95,1,5,11,WSW,25.0,NaN,NaN,54.2,0.4,130.0,3.0,NaN
104,Apr 19 22:00,18,95,2,6,9,W,25.0,NaN,NaN,54.1,0.3,130.0,3.0,NaN


Read the danger ratings

In [64]:
def get_seasonal_danger():
    return pd.read_csv('../content/csv/danger_ratings.csv')

def remove_no_rating(danger_df):
    return danger_df[danger_df['Danger Rating'] != 'NO RATING']

seasonal_danger = get_seasonal_danger()
seasonal_danger = remove_no_rating(seasonal_danger)

def select_danger_area(danger_df, target_area='West Slopes South'):
    # only select target area
    return danger_df[(danger_df == target_area).any(axis=1)]

def map_dangerrating_to_number(danger_df):
    danger_number = { 'NO RATING': 0, 'LOW': 1, 'MODERATE': 2, 'CONSIDERABLE': 3, 'HIGH': 4, 'EXTREME': 5 }
    danger_df['Danger Rating'] = danger_df['Danger Rating'].map(danger_number)
    return danger_df

seasonal_danger = select_danger_area(seasonal_danger)
seasonal_danger = map_dangerrating_to_number(seasonal_danger)
seasonal_danger.head()

,Date,Area,Danger Rating
15,Apr 15 2023,West Slopes South,2
27,Apr 14 2023,West Slopes South,1
28,Apr 13 2023,West Slopes South,2
43,Apr 12 2023,West Slopes South,2
54,Apr 11 2023,West Slopes South,2


Wind direction -> number

In [65]:
def map_winddirection_to_number(weather_df):
    weather_number = { 
        'N': 0, 'NNE': 1, 'NE': 2, 'ENE': 3,
        'E': 4, 'ESE': 5, 'SE': 6, 'SSE': 7,
        'S': 8, 'SSW': 9, 'SW': 10, 'WSW': 11,
        'W': 12, 'WNW': 13, 'NW': 14, 'NNW': 15
    }
    weather_df['Dir deg'] = weather_df['Dir deg'].map(weather_number)
    return weather_df

seasonal_weather = map_winddirection_to_number(seasonal_weather)
seasonal_weather.iloc[1000:1005]

,Date PDT,Temp F NWAC,RH %,Min mph,Spd mph,Gust mph,Dir deg,Temp F SNOTEL,PcpAc in,∆Pcp in,SWE in,∆SWE in,SnoHt in,∆SnoHt in,Temp F
1000,Mar 13 14:00,28,98,1,7,13,12,33.0,48.1,0.2,44.1,1.5,122.0,9.0,NaN
1001,Mar 13 13:00,30,99,0,4,11,13,33.0,48.1,0.2,44.0,1.4,121.0,9.0,NaN
1002,Mar 13 12:00,30,98,0,7,24,10,33.0,48.1,0.1,43.9,1.3,121.0,8.0,NaN
1003,Mar 13 11:00,30,99,3,11,35,9,33.0,48.1,0.2,43.8,1.2,120.0,7.0,NaN
1004,Mar 13 10:00,30,98,4,11,35,9,33.0,48.1,0.1,43.8,1.2,120.0,7.0,NaN


Connect aggregated weather data to danger ratings

Generate metrics

In [66]:
# --- Start Validated
def get_one_day_ago(days_df):
    # seems to work correctly
    return days_df.iloc[0:24]
def get_two_days_ago(days_df):
    # seems to work correctly
    return days_df.iloc[24:24*2]
def get_three_days_ago(days_df):
    # seems to work correctly
    return days_df.iloc[24*2:24*3]
def get_four_days_ago(days_df):
    # seems to work correctly
    return days_df.iloc[24*3:24*4]
def get_past_three_days(days_df):
    return days_df.iloc[0:24*3]
def get_max_air_temp(day):
    return day['Temp F'].max()
def get_total_snowfall(day):
    # todo: 0 if negative?
    total = day['∆SnoHt in'].iat[0]
    # return total if total > 0 else 0
    return total
def get_max_wind(day):
    return day['Spd mph'].max() # should we use gust?
def get_weighted_snowfall(days_df):
    one = get_one_day_ago(days_df)
    # todo: check
    two = get_two_days_ago(days_df)
    three = get_three_days_ago(days_df)
    four = get_four_days_ago(days_df)
    return get_total_snowfall(one) * 1.0 \
        + get_total_snowfall(two) * 0.75 \
        + get_total_snowfall(three) * 0.5 \
        + get_total_snowfall(four) * 0.25
def get_diff_min_temp(earlier_day, later_day):
    return later_day['Temp F'].min() - earlier_day['Temp F'].min()
def get_heavy_snowfall(day):
    """Get whether the day had >12in of snow accumulation or >1in/hr of snow accumulation for 6+ hrs"""
    # todo: day vs hour separate metrics?
    if day['∆SnoHt in'].max() >= 12:
        return 1
    # num_hrs_over_6in_accum = day['']
    # Count hourly deltas
    count = 0
    last = day['SnoHt in'].iat[0]
    for ht in day['SnoHt in'][1:]:
        if ht - last >= 1:
            count += 1
        last = ht
    return 1 if count >= 6 else 0
def get_high_winds(day):
    """Get whether the wind speed exceeded 20 mph at any point"""
    # todo: try using gust?
    # todo: 25mph threshold?
    if day['Spd mph'].max() > 20:           
        return 1
    return 0
def get_sum_max_temp_72(day_one, day_two, day_three):
    return get_max_air_temp(day_one) + get_max_air_temp(day_two) + get_max_air_temp(day_three)
def get_daily_swe(day):
    return day['∆SWE  in'].iat[11] # return from the middle of the day
# --- End Validated


Validate metrics

In [67]:
def validate_metrics():

    # --- Initialize
    seasonal_nwac = get_seasonal_nwac()
    seasonal_nwac = seasonal_nwac.fillna(0)
    seasonal_nwac = map_winddirection_to_number(seasonal_nwac)
    seasonal_snotel = get_seasonal_snotel()
    seasonal_snotel = seasonal_snotel.fillna(0)
    seasonal_weather = get_seasonal_weather(seasonal_nwac, seasonal_snotel)
    # --- Done

    four_random_days = seasonal_weather.iloc[100:100+24*4] # todo: this will break when changing weather df
    assert four_random_days.shape[0] == 96, 'Make sure that I am correctly indexing'
    assert four_random_days.iloc[0]['Date PDT'] == 'Apr 20 2:00', 'Make sure this date starts where I think it does'
    day = get_one_day_ago(four_random_days)

    # some of these are kind of stupid tests, but it is probably a good idea
    #   to write them to ensure that nothing changes
    def validate_get_one_day_ago():
        day = get_one_day_ago(four_random_days)
        assert day.shape[0] == 24, 'Make sure indexing is correct'
        assert day.iloc[0]['Date PDT'] == 'Apr 20 2:00', 'Make sure the start time is correct'
        assert day.iloc[-1]['Date PDT'] == 'Apr 19 3:00', 'Make sure the end time is correct'
    def validate_get_two_days_ago():
        day = get_two_days_ago(four_random_days)
        assert day.shape[0] == 24, 'Make sure indexing is correct'
        assert day.iloc[0]['Date PDT'] == 'Apr 19 2:00', 'Make sure the start time is correct'
        assert day.iloc[-1]['Date PDT'] == 'Apr 18 3:00', 'Make sure the end time is correct'
    def validate_get_three_days_ago():
        day = get_three_days_ago(four_random_days)
        assert day.shape[0] == 24, 'Make sure indexing is correct'
        assert day.iloc[0]['Date PDT'] == 'Apr 18 2:00', 'Make sure the start time is correct'
        assert day.iloc[-1]['Date PDT'] == 'Apr 17 3:00', 'Make sure the end time is correct'
    def validate_get_four_days_ago():
        day = get_four_days_ago(four_random_days)
        assert day.shape[0] == 24, 'Make sure indexing is correct'
        assert day.iloc[0]['Date PDT'] == 'Apr 17 2:00', 'Make sure the start time is correct'
        assert day.iloc[-1]['Date PDT'] == 'Apr 16 3:00', 'Make sure the end time is correct'
    def validate_get_past_three_days():
        day = get_past_three_days(four_random_days)
        assert day.shape[0] == 24*3, 'Make sure indexing is correct'
        assert day.iloc[0]['Date PDT'] == 'Apr 20 2:00', 'Make sure the start time is correct'
        assert day.iloc[-1]['Date PDT'] == 'Apr 17 3:00', 'Make sure the end time is correct'
    def validate_get_max_air_temp():    
        assert get_max_air_temp(day) == 31, 'Incorrect max air temp'
    def validate_get_total_snowfall():
        assert get_total_snowfall(day) == 2, 'Incorrect total snowfall'
    def validate_get_max_wind():
        assert get_max_wind(day) == 25, 'Incorrect max windspeed'
    def validate_get_weighted_snowfall():
        pass # This seems dumb to check
    def validate_get_diff_min_temp():
        assert get_diff_min_temp(get_two_days_ago(four_random_days), day) == 19.5-17.5, 'Incorrect difference in minimum daily temperature'
    def validate_get_heavy_snowfall():
        assert get_heavy_snowfall(day) == 1, 'Incorrect heavy snowfall'
    def validate_get_high_winds():
        # This also seems dumb to check
        assert get_high_winds(day) == 1, 'Incorrect high winds'
    def validate_get_sum_max_temp_72():
        assert False, 'Undefined validation function'
    def validate_get_daily_swe():
        assert False, 'Undefined validation function'

    def validate_all():
        validate_get_one_day_ago()
        validate_get_two_days_ago()
        validate_get_three_days_ago()
        validate_get_four_days_ago()
        validate_get_past_three_days()
        validate_get_max_air_temp() # not working?
        validate_get_total_snowfall()
        validate_get_max_wind()
        validate_get_weighted_snowfall()
        validate_get_diff_min_temp()
        validate_get_heavy_snowfall()
        validate_get_high_winds()
        # validate_get_sum_max_temp_72()
        # validate_get_daily_swe()

    validate_all()
# validate_metrics()

In [78]:
import math

def setup_training_data(seasonal_df):
    training_data = pd.DataFrame([], columns=seasonal_df.columns)
    # Add columns for the new metrics
    training_data['Max Air Temp 24hr'] = None
    training_data['Max Air Temp 72hr'] = None
    training_data['Total Snowfall 24hr'] = None
    training_data['Total Snowfall 72hr'] = None
    training_data['Max Windspeed 24hr'] = None
    training_data['Weighted Snowfall 96hr'] = None
    training_data['Min Temp Diff 48hr'] = None
    training_data['Was Heavy Snowfall 24hr'] = None
    training_data['Was High Winds 24hr'] = None
    training_data['Sum Max Temp 72hr'] = None
    training_data['Delta SWE 24hr'] = None
    # Add column for danger rating
    training_data['Yesterday Danger'] = None
    training_data['Danger Rating'] = None
    return training_data

def initialize_dataframes():
    """Initialize necessary dataframes, return seasonal, danger, result"""
    seasonal_nwac = get_seasonal_nwac()
    seasonal_nwac = seasonal_nwac.fillna(0)
    seasonal_nwac = map_winddirection_to_number(seasonal_nwac)

    seasonal_snotel = get_seasonal_snotel()
    seasonal_snotel = seasonal_snotel.fillna(0)

    seasonal_weather = get_seasonal_weather(seasonal_nwac, seasonal_snotel)

    seasonal_danger = get_seasonal_danger()
    seasonal_danger = remove_no_rating(seasonal_danger)
    seasonal_danger = select_danger_area(seasonal_danger)
    seasonal_danger = map_dangerrating_to_number(seasonal_danger)

    training_data = setup_training_data(seasonal_weather)

    return seasonal_weather, seasonal_danger, training_data

def connect_danger_to_aggregated_weather(seasonal_df, danger_df, result_df):
    def get_days_past(seasonal_df, days=4):
        return seasonal_df.iloc[idx+1:idx-1+24*days]
    
    def get_date_and_datestart(row):
        # Get the date (Month, day)
        date = row['Date'].split(' ')[0] + ' ' + row['Date'].split(' ')[1] # BAD
        # Get the time at the start of the day
        date_start = date + ' 0:00'
        return date, date_start
    
    def get_idx(seasonal_df, date_start):
        return seasonal_df[(seasonal_df == date_start).any(axis=1)].index[0]

    def aggregate(days_df):
        assert days_df is not None, 'Days must be initialized'

        past_day = get_one_day_ago(days_df)
        past_three_days = get_past_three_days(days_df)
        max_temp_24 = get_max_air_temp(past_day)
        max_temp_72 = get_max_air_temp(past_three_days)
        snowfall_24 = get_total_snowfall(past_day)
        snowfall_72 = snowfall_24 + get_total_snowfall(get_two_days_ago(days_df)) \
            + get_total_snowfall(get_three_days_ago(days_df))
        max_wind_24 = get_max_wind(past_day)
        weighted_snowfall = get_weighted_snowfall(days_df)
        diff_min_temp = get_diff_min_temp(days_df.iloc[24*2:24*3], past_day)
        heavy_snowfall = get_heavy_snowfall(past_three_days)
        high_winds = get_high_winds(past_day)
        sum_max_temp_72 = get_sum_max_temp_72(past_day, get_two_days_ago(days_df), get_three_days_ago(days_df))
        daily_swe = get_daily_swe(past_day)

        days_df = days_df.drop('Date PDT', axis=1).mean()

        days_df['Max Air Temp 24hr'] = max_temp_24
        days_df['Max Air Temp 72hr'] = max_temp_72
        days_df['Total Snowfall 24hr'] = snowfall_24
        days_df['Total Snowfall 72hr'] = snowfall_72
        days_df['Max Windspeed 24hr'] = max_wind_24
        days_df['Weighted Snowfall 96hr'] = weighted_snowfall
        days_df['Min Temp Diff 48hr'] = diff_min_temp
        days_df['Was Heavy Snowfall 24hr'] = heavy_snowfall
        days_df['Was High Winds 24hr'] = high_winds
        days_df['Sum Max Temp 72hr'] = sum_max_temp_72
        days_df['Delta SWE 24hr'] = daily_swe

        return days_df
    
    def add_and_reindex(result_df, row):
        result_df.loc[-1] = row
        result_df.index = result_df.index+1
        return result_df
    
    def add_date_and_danger(row, date, danger, prev_danger):
        row['Date PDT'] = date
        row['Yesterday Danger'] = prev_danger
        row ['Danger Rating'] = danger
        return row
    
    prev_day_danger = 0
    # For every danger rating (ie for every day)
    for idx, row in danger_df.iterrows():
        date, date_start = get_date_and_datestart(row)
        # Use this to index the seasonal dataframe
        idx = get_idx(seasonal_df, date_start)
        # Get the past x days
        days_past = get_days_past(seasonal_df)
        # Aggregate them
        agg_past = aggregate(days_past)
        # Tag with date and danger rating
        agg_past = add_date_and_danger(agg_past, date, row['Danger Rating'], prev_day_danger)
        # Add back into the dataframe
        result_df = add_and_reindex(result_df, agg_past)
        # Cache today's danger to use as previous next iter
        prev_day_danger = 0 #row['Danger Rating']
    # Drop a BUNCH of columns
    # cols = [1,2,3,4,5,6,7,8,9, 10,11,12] # BAD
    # result_df.drop(result_df.columns[cols], axis=1, inplace=True)
    result_df = result_df.fillna(0)
    return result_df

seasonal_weather, seasonal_danger, training_data = initialize_dataframes()

training_data = connect_danger_to_aggregated_weather(seasonal_weather, seasonal_danger, training_data)
training_data.iloc[20:25]

/tmp/ipykernel_27752/1993969672.py:12: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  mean_temp = seasonal_weather.drop(['Temp F NWAC', 'Temp F SNOTEL'], axis=1).mean()


Error: "['Date'] not found in axis"


,Date PDT,Temp F NWAC,RH %,Min mph,Spd mph,Gust mph,Dir deg,Temp F SNOTEL,PcpAc in,∆Pcp in,...,Total Snowfall 72hr,Max Windspeed 24hr,Weighted Snowfall 96hr,Min Temp Diff 48hr,Was Heavy Snowfall 24hr,Was High Winds 24hr,Sum Max Temp 72hr,Delta SWE 24hr,Yesterday Danger,Danger Rating
125,Mar 26,19.659574,88.404255,5.159574,12.329787,21.861702,10.574468,26.638298,53.068085,0.387234,...,3.0,15.0,0.50,0,1.0,0.0,0,0.0,0,2
124,Mar 25,22.510638,87.191489,6.276596,14.223404,25.148936,8.723404,28.989362,52.653191,0.429787,...,1.0,20.0,0.75,0,1.0,0.0,0,0.0,0,2
123,Mar 24,25.446809,87.382979,6.138298,13.127660,22.425532,8.478723,31.361702,52.196809,0.274468,...,3.0,19.0,3.75,0,1.0,0.0,0,0.8,0,3
122,Mar 23,27.340426,85.212766,8.297872,15.244681,24.265957,6.957447,31.925532,51.922340,0.153191,...,-1.0,32.0,-2.00,0,1.0,1.0,0,-0.1,0,3
121,Mar 22,28.212766,90.531915,8.425532,15.414894,23.968085,6.840426,32.372340,51.769149,0.603191,...,-1.0,32.0,-1.00,0,1.0,1.0,0,0.2,0,2


Clean up NaN's

Replacing with 0 for now, but todo: is there a better way?

In [79]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor # try classifier next

def run_model(training_data):

    # date isn't a number, and isn't relevant (yet)
    try:
        training_data = training_data.drop('Date PDT', axis=1)
    except:
        assert False, 'Undefined validation function'

    drop = 'Danger Rating'
    x = training_data.drop(drop, axis=1)
    y = training_data[drop]

    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

    rf_regressor = RandomForestRegressor(random_state=42)

    rf_regressor.fit(x_train, y_train)

    y_pred_train = rf_regressor.predict(x_train)
    print('Training R2 score:', r2_score(y_train, y_pred_train))

    y_pred = rf_regressor.predict(x_test)
    print('Actual R2 score:', r2_score(y_test, y_pred))
run_model(training_data)

Training R2 score: 0.8483577291037261
Actual R2 score: 0.10542824267782447


Getting better, but still not there yet

Try using RandomizedSearchCV to come up with better hyperparameters

In [80]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor # try classifier next

from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start=200, stop=2000, num=10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in the tree
max_depth = [int(x) for x in np.linspace(10, 110, num=11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2,5,10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1,2,4]
# Method of selecting samples for training each tree
bootstrap = [True,False]

# Create random grid
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

# date isn't a number, and isn't relevant (yet)
try:
    training_data = training_data.drop('Date PDT', axis=1)
except:
    assert False, 'Undefined validation function'

drop = 'Danger Rating'
x = training_data.drop(drop, axis=1)
y = training_data[drop]

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

rf_regressor = RandomForestRegressor(random_state=0)

rf = RandomForestRegressor()
"""
rf_random = RandomizedSearchCV( \
    estimator=rf, param_distributions=random_grid, n_iter=100, \
    cv=3, verbose=2, random_state=42, n_jobs=-1)
rf_random.fit(x_train, y_train)
"""
# pprint(rf_random.best_params_)

rf_regressor.fit(x_train, y_train)

y_pred_train = rf_regressor.predict(x_train)
r2_score(y_train, y_pred_train)


0.868346550855992

In [81]:

best_params = {
    'bootstrap': True,
    'max_depth': 10,
    'max_features': 'sqrt',
    'min_samples_leaf': 4,
    'min_samples_split': 2,
    'n_estimators': 1600
}
rf_test = RandomForestRegressor(bootstrap=True, max_depth=10, max_features='sqrt', \
                           min_samples_leaf=4, min_samples_split=2, n_estimators=1600, random_state=42)

rf_test.fit(x_train, y_train)

y_pred_train = rf_test.predict(x_train)
r2_score(y_train, y_pred_train)

0.5252331795818823

In [82]:
y_pred = rf_test.predict(x_test)
r2_score(y_test, y_pred)

0.06120164842407427